## PyTorch Workflow

Let's explore a an example PyTorch end-to-end workflow.

In [2]:
what_were_covering = {1: "data (prepare and load)",
                      2: "Build the model",
                      3: "Fitting the model to data (training)",
                      4: "making predictions and evaluating a model (inference)",
                      5: "Saving and loading a model",
                      6: "Putting it all together"}
what_were_covering

{1: 'data (prepare and load)',
 2: 'Build the model',
 3: 'Fitting the model to data (training)',
 4: 'making predictions and evaluating a model (inference)',
 5: 'Saving and loading a model',
 6: 'Putting it all together'}

In [3]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.1.2+cu121'

## Data (preparing and loading)